# GPU Testing Notebook

In [1]:
!nvidia-smi

Fri Nov 11 16:24:32 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.85.02    Driver Version: 510.85.02    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GRID A100X-8C       On   | 00000000:00:06.0 Off |                    0 |
| N/A   N/A    P0    N/A /  N/A |   1127MiB /  8192MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Tensorflow Check

In [2]:
import tensorflow as tf

2022-11-11 16:24:33.085171: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-11 16:24:33.223312: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-11 16:24:33.893427: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvrtc.so.11.0: cannot open shared object file: No such file or directory
2022-11-11 16:24:33.893605: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvrtc.so.11.0: cannot open shared object file: No such file or direc

In [3]:
tf.config.list_physical_devices('GPU')

2022-11-11 16:24:34.754942: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-11 16:24:34.759597: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-11 16:24:34.759748: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [21]:
is_cuda_gpu_available = tf.config.list_physical_devices('GPU')
is_cuda_gpu_available

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

## PyTorch Check

In [5]:
import torch

In [6]:
torch.cuda.is_available()

True

In [7]:
torch.cuda.device_count()

1

In [8]:
torch.cuda.device(0)

In [9]:
torch.cuda.get_device_name(0)

'GRID A100X-8C'

## XGBoost Check

In [10]:
import xgboost as xgb

# Dataset we will load in
from sklearn.datasets import load_iris

In [11]:
iris = load_iris()
X = iris["data"]
y = iris["target"]
xgb_model = xgb.XGBRegressor(
        # If there is no GPU, the tree_method kwarg will cause either
        # - an error in `xgb_model.fit(X, y)` (seen with pytest) or
        # - a warning printed to the console (seen in Spyder)
        # It's unclear which of the two happens under what circumstances.
        tree_method="gpu_hist" # GPU!!
    )
xgb_model.fit(X, y)
# Check that no warning was printed.

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=0, gpu_id=0,
             grow_policy='depthwise', importance_type=None,
             interaction_constraints='', learning_rate=0.300000012, max_bin=256,
             max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
             max_depth=6, max_leaves=0, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, ...)

## Catboost Check

In [12]:
import catboost

from catboost.utils import get_gpu_device_count
from catboost import CatBoostRegressor

In [13]:
catboost.__version__

'1.1.1'

In [14]:
print('I see %i GPU devices' % get_gpu_device_count())

I see 1 GPU devices


Below seems like a catboost issue, not a GPU one

In [15]:
model = CatBoostRegressor(iterations=1000,
                          task_type="GPU",
                          gpu_ram_part=0.5
                         # n_models = 0.05
                         # devices='1'
                         )
model.fit(X,
          y,
          verbose=False)

## CuPy

In [16]:
# !pip install cupy-cuda11x

In [17]:
import cupy as cp

In [18]:
x = cp.arange(6).reshape(2, 3).astype('f')

In [19]:
x

array([[0., 1., 2.],
       [3., 4., 5.]], dtype=float32)

In [20]:
x.sum(axis=1)

array([ 3., 12.], dtype=float32)